In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
# Load the dataset
# df = pd.read_excel('../../dataset/Dataset_IoV.xlsx')  
df = pd.read_csv('..//..//dataset//preliminary_dataset.csv')  


# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']
# scaler=RobustScaler()
# X_scaled = scaler.fit_transform(X)

# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)
X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [3]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(80000, 1, 6) (80000,) (100000, 1, 6)


# Model

In [4]:
def build_lstm():
    model = Sequential()
    
    # Input layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    # LSTM layer with 3 hidden layers
    model.add(LSTM(units=100, activation='relu', return_sequences=True))
    model.add(LSTM(units=100,activation='relu', return_sequences=True))
    model.add(LSTM(units=100,))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model


In [5]:
# Instantiate and train the model
model = build_lstm()
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 0.3007 - mae: 0.2189 - mse: 0.1029 - val_loss: 6.3349e-04 - val_mae: 6.1689e-04 - val_mse: 2.8345e-05
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0059 - mae: 0.0020 - mse: 0.0012 - val_loss: 3.8820e-04 - val_mae: 3.8693e-04 - val_mse: 2.4811e-06
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 5.6888e-04 - mae: 4.5700e-04 - mse: 7.0911e-05 - val_loss: 2.2396e-04 - val_mae: 2.2357e-04 - val_mse: 7.6944e-07
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 4.0219e-04 - mae: 3.2342e-04 - mse: 5.1824e-05 - val_loss: 1.3037e-04 - val_mae: 1.3023e-04 - val_mse: 2.8234e-07
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 3.4576e-04 - mae: 2.1817e-04 - mse: 3.1399e-05 - val_loss: 8.3699e-05 - val_mae: 8.3625e-05 - val_mse: 1.4829e-07
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 1.3339e-04 - mae: 1.1170e-04 - mse: 6.9463e-06 - val_loss: 5.6172e-05 - val_mae: 5.6134e-05 

In [6]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [7]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')


Mean Squared Error for y_pred (MSE): 0.0000142457574839
Root Mean Squared Error (RMSE): 0.0037743552408163


In [8]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [9]:
import time

# Start time
start_time = time.time()

# Make predictions
y_pred = model.predict(X_test)

# End time
end_time = time.time()

# Total AES time
total_time = end_time - start_time  

# AES per sample
aes_per_sample = (total_time / len(X_test)) * 1e6  # Convert to microseconds (µs)

print(f'Total AES Time: {total_time:.6f} seconds')
print(f'AES Time per Sample: {aes_per_sample:.2f} µs/sample')


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Total AES Time: 1.687992 seconds
AES Time per Sample: 84.40 µs/sample
